## 1. Import Package
- 问题点，在jupyter使用tf调试代码时候记得要清空缓存，重新开始，不然似乎会出现网络重定义的行为

In [1]:
import tensorflow as tf
import os
import tensorflow.contrib.slim as slim
from create_record_files import get_example_nums
import PARAMS as Param
from models import dense_net_b3_k12_se as net
import dataset_factory.dataset_factory as datasets
from tensorflow.python.ops import control_flow_ops
import matplotlib.pyplot as plt

%matplotlib inline

## 2. define train -parameters

In [2]:
PARAMS = Param.Params()
BATCH_SIZE = 32
NUM_EPOCHS = 300
NUM_CLASS = 3
WEIGHT_DECAY = 1e-5
KEEP_PROB = 0.9
BASE_LR1 = 1e-3
BASE_LR2 = 5e-4
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
IMAGE_DEPTH = 3
DECAY_EPOCH = 8  # learning_rate 衰减的时间
TENSORBOARD_PATH = PARAMS.params['path']['train_tensorBoardPath']

## 3. define file path

In [3]:
validation_index = 5
train_dir = "dataset/record_file/iwatch_224_record_v{0}/train".format(validation_index)
print(train_dir)
train_num = get_example_nums(train_dir)
print("train num = {0}".format(train_num))
val_dir = "dataset/record_file/iwatch_224_record_v{0}/val".format(validation_index)
val_num = get_example_nums(val_dir)
print("validation num = {0}".format(val_num))

dataset/record_file/iwatch_224_record_v5/train
['train-00-of-00']
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
train num = 439
['val-00-of-00']
validation num = 110


## 4. define dataset

In [4]:
# print_tensors_in_checkpoint_file(MODEL_PATH, None, False, True)
#定义数据集
training_dataset = datasets.get_dataset('iwatch', train_dir,
                                        'train', batch_size=BATCH_SIZE,label_num=3)
# train_dataset 用epochs控制循环
validation_dataset = datasets.get_dataset('iwatch', val_dir,
                                        'test', batch_size=BATCH_SIZE, label_num=3)

train_iterator = training_dataset.make_initializable_iterator()
# make_initializable_iterator 每个epoch都需要初始化
val_iterator = validation_dataset.make_initializable_iterator()
# make_one_shot_iterator不需要初始化，根据需要不停循环
train_images, train_labels = train_iterator.get_next()
val_images, val_labels = val_iterator.get_next()

['dataset/record_file/iwatch_224_record_v5/train\\train-00-of-00']
Instructions for updating:
Colocations handled automatically by placer.
['dataset/record_file/iwatch_224_record_v5/val\\val-00-of-00']


## 5. define models
 - define input placehoder
 - define propagate
 - define loss-function
 - define train_op
 - plot feature-map

In [5]:
# 定义输入
with tf.name_scope("inputs"):
    is_training = tf.placeholder(tf.bool)
    images = tf.placeholder(
        dtype=tf.float32, shape=[None, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH], name='inputs')
    labels = tf.placeholder(dtype=tf.int32, shape=[None, NUM_CLASS], name='label')

with tf.name_scope('nets'):
    logits, endPoints = net.inference(inputs=images, num_classes=NUM_CLASS,
                                     is_training=is_training, dropout_keep_prob = KEEP_PROB)

with tf.name_scope("loss_function"):
    tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
    loss_entro = tf.losses.get_losses()
    loss_entro_mean = tf.reduce_mean(loss_entro)
    tf.summary.scalar("cross_entropy", loss_entro_mean)
# prediction

with tf.name_scope("train_op"):
    # fc8_optimizer = tf.train.GradientDescentOptimizer(BASE_LR1)  不能定义太多优化器，内存会爆掉
    global_step = tf.train.create_global_step()
    learning_rate = tf.train.exponential_decay(
        PARAMS.params['model']['baseLR'],
        global_step,
        DECAY_EPOCH*train_num / BATCH_SIZE,
        PARAMS.params['model']['decayLR'])

    optimizer = tf.train.AdamOptimizer(learning_rate)

    full_train_op = slim.learning.create_train_op(loss_entro_mean,
                                                  optimizer,
                                                  global_step=global_step)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
#         print("BN parameters: ", update_ops)
        updates = tf.group(*update_ops)
        full_train_op = control_flow_ops.with_dependencies([updates], full_train_op)
#     for v in tf.all_variables():
#         print(v.name)
#         if 'batch_normalization' in v.name:
#             tf.summary.histogram(v.name, v)

    # 冻结fc7 以前的层
    # fc8_train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step, var_list=fc8_variables)
    # 全部训练

with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

with tf.name_scope("input"):
    image_shape_input = tf.reshape(images, [-1, 224, 224, 1])
    tf.summary.image('input_image', image_shape_input, 10)

with tf.name_scope("feature_map"):
    feture_maps = endPoints["layer1"]
    # 取其中的第一张图像
    # TODO 第一层输出的feauture值可能会不一样
    # TODO ix，iy为宽度和高度
    # TODO cx cy为每行显示的图像 ix*iy=channels
    feture_maps = tf.slice(feture_maps, (0, 0, 0, 0), (1, -1, -1, -1))
    ix = iy = 112
    ix += 4
    iy += 4
    #类似subplot的方法图像显示为4x4
    cy = 4
    cx = 3
    #做padding方便将所有图分开
    feture_maps = tf.image.resize_image_with_crop_or_pad(feture_maps, iy, ix)
    # reshape 成 56x56x4x4
    feture_maps = tf.reshape(feture_maps, (iy, ix, cy, cx))
    #交换维度
    feture_maps = tf.transpose(feture_maps, (2, 0, 3, 1))  # cy,iy,cx
    #将所有通道的图片组成一张图片显示
    feture_maps = tf.reshape(feture_maps, (1, cy * iy, cx * ix, 1))

    tf.summary.image("layer1_feature_map", feture_maps, 1)

init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]
saver = tf.train.Saver()
summary_merge = tf.summary.merge_all()

# 结束当前的计算图，使之成为只读
#     tf.get_default_graph().finalize()

Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


### 6. begin training

In [6]:
def train():
    acc_list = []
    with tf.Session() as sess:
        # 先初始化网络
        sess.run(init_op)
        tensorboard_writer = tf.summary.FileWriter(TENSORBOARD_PATH, sess.graph)
        max_acc = 0.70
        for epoch in range(NUM_EPOCHS):
            print('Starting epoch %d / %d' % (epoch, NUM_EPOCHS))
            sess.run(train_iterator.initializer)
            while True:
                try:
                    train_batch_images, train_batch_labels \
                        = sess.run([train_images, train_labels])
                    _,summary, train_loss, train_acc = sess.run([full_train_op, summary_merge,loss_entro_mean, accuracy],
                                                         feed_dict={is_training: True,
                                                                images: train_batch_images,
                                                                labels: train_batch_labels})
                    step = sess.run(global_step)
                    tensorboard_writer.add_summary(summary,step)
                    if step %10 == 0:
                        print("epoch = {0} step = {3}: train-loss = {1}, batch-acc = {2}".format(epoch, train_loss,train_acc,step))
                except tf.errors.OutOfRangeError:
                    break
            sess.run(val_iterator.initializer)
            num_correct = 0
            while True:
                try:
                    val_batch_images, val_batch_labels \
                        = sess.run([val_images, val_labels])

                    correct_pred = sess.run(correct_prediction, feed_dict={
                                                        is_training: False,
                                                        images: val_batch_images,
                                                        labels: val_batch_labels})

                    num_correct += correct_pred.sum()
                except tf.errors.OutOfRangeError:
                    break
            val_acc = float(num_correct) / val_num
            acc_list.append(val_acc)
            # Plot learning curve (with costs)
            if val_acc >= max_acc or epoch > NUM_EPOCHS-2:
                max_acc = val_acc
                best_models = os.path.join(PARAMS.params['path']['model_path'],
                                           'model_epoch{}_{:.4f}.ckpt'.format(epoch, val_acc))
                saver.save(sess,best_models)
            print("epoch = {0}, val-acc = {1}".format(epoch, val_acc))
        tensorboard_writer.close()
        return acc_list

In [7]:
if __name__ == '__main__':
    acc_list = train()

Starting epoch 0 / 300
epoch = 0 step = 10: train-loss = 1.019791603088379, batch-acc = 0.40625
epoch = 0, val-acc = 0.2818181818181818
Starting epoch 1 / 300
epoch = 1 step = 20: train-loss = 1.0328987836837769, batch-acc = 0.34375
epoch = 1, val-acc = 0.2818181818181818
Starting epoch 2 / 300
epoch = 2 step = 30: train-loss = 1.0051724910736084, batch-acc = 0.46875
epoch = 2 step = 40: train-loss = 1.117214560508728, batch-acc = 0.3125
epoch = 2, val-acc = 0.2818181818181818
Starting epoch 3 / 300
epoch = 3 step = 50: train-loss = 1.0331687927246094, batch-acc = 0.46875
epoch = 3, val-acc = 0.2818181818181818
Starting epoch 4 / 300
epoch = 4 step = 60: train-loss = 0.8802303075790405, batch-acc = 0.65625
epoch = 4 step = 70: train-loss = 0.9815853238105774, batch-acc = 0.52173912525177
epoch = 4, val-acc = 0.2818181818181818
Starting epoch 5 / 300
epoch = 5 step = 80: train-loss = 0.8540412783622742, batch-acc = 0.59375
epoch = 5, val-acc = 0.2818181818181818
Starting epoch 6 / 300
e

epoch = 48, val-acc = 0.7636363636363637
Starting epoch 49 / 300
epoch = 49 step = 690: train-loss = 0.2524052560329437, batch-acc = 0.96875
epoch = 49 step = 700: train-loss = 0.227411150932312, batch-acc = 0.95652174949646
epoch = 49, val-acc = 0.7545454545454545
Starting epoch 50 / 300
epoch = 50 step = 710: train-loss = 0.19144755601882935, batch-acc = 1.0
epoch = 50, val-acc = 0.7818181818181819
Starting epoch 51 / 300
epoch = 51 step = 720: train-loss = 0.17967887222766876, batch-acc = 1.0
Instructions for updating:
Use standard file APIs to delete files with this prefix.
epoch = 51, val-acc = 0.8090909090909091
Starting epoch 52 / 300
epoch = 52 step = 730: train-loss = 0.1879434734582901, batch-acc = 1.0
epoch = 52 step = 740: train-loss = 0.2676052153110504, batch-acc = 0.9375
epoch = 52, val-acc = 0.6818181818181818
Starting epoch 53 / 300
epoch = 53 step = 750: train-loss = 0.22257111966609955, batch-acc = 0.96875
epoch = 53, val-acc = 0.7181818181818181
Starting epoch 54 / 

epoch = 95 step = 1340: train-loss = 0.09118407964706421, batch-acc = 1.0
epoch = 95, val-acc = 0.7
Starting epoch 96 / 300
epoch = 96 step = 1350: train-loss = 0.09959572553634644, batch-acc = 1.0
epoch = 96, val-acc = 0.8
Starting epoch 97 / 300
epoch = 97 step = 1360: train-loss = 0.15722985565662384, batch-acc = 0.96875
epoch = 97 step = 1370: train-loss = 0.09012533724308014, batch-acc = 1.0
epoch = 97, val-acc = 0.8181818181818182
Starting epoch 98 / 300
epoch = 98 step = 1380: train-loss = 0.12572641670703888, batch-acc = 0.96875
epoch = 98, val-acc = 0.7545454545454545
Starting epoch 99 / 300
epoch = 99 step = 1390: train-loss = 0.15660496056079865, batch-acc = 0.9375
epoch = 99 step = 1400: train-loss = 0.08066313713788986, batch-acc = 1.0
epoch = 99, val-acc = 0.7454545454545455
Starting epoch 100 / 300
epoch = 100 step = 1410: train-loss = 0.11099180579185486, batch-acc = 1.0
epoch = 100, val-acc = 0.7818181818181819
Starting epoch 101 / 300
epoch = 101 step = 1420: train-lo

epoch = 143, val-acc = 0.8272727272727273
Starting epoch 144 / 300
epoch = 144 step = 2020: train-loss = 0.07589707523584366, batch-acc = 1.0
epoch = 144 step = 2030: train-loss = 0.056630346924066544, batch-acc = 1.0
epoch = 144, val-acc = 0.8545454545454545
Starting epoch 145 / 300
epoch = 145 step = 2040: train-loss = 0.06719879806041718, batch-acc = 1.0
epoch = 145, val-acc = 0.8363636363636363
Starting epoch 146 / 300
epoch = 146 step = 2050: train-loss = 0.10326430946588516, batch-acc = 1.0
epoch = 146, val-acc = 0.8272727272727273
Starting epoch 147 / 300
epoch = 147 step = 2060: train-loss = 0.12352955341339111, batch-acc = 1.0
epoch = 147 step = 2070: train-loss = 0.055006686598062515, batch-acc = 1.0
epoch = 147, val-acc = 0.8454545454545455
Starting epoch 148 / 300
epoch = 148 step = 2080: train-loss = 0.062332313507795334, batch-acc = 1.0
epoch = 148, val-acc = 0.8636363636363636
Starting epoch 149 / 300
epoch = 149 step = 2090: train-loss = 0.08312539011240005, batch-acc =

epoch = 191, val-acc = 0.8181818181818182
Starting epoch 192 / 300
epoch = 192 step = 2690: train-loss = 0.051885850727558136, batch-acc = 1.0
epoch = 192 step = 2700: train-loss = 0.05049509555101395, batch-acc = 1.0
epoch = 192, val-acc = 0.8636363636363636
Starting epoch 193 / 300
epoch = 193 step = 2710: train-loss = 0.05257371440529823, batch-acc = 1.0
epoch = 193, val-acc = 0.8363636363636363
Starting epoch 194 / 300
epoch = 194 step = 2720: train-loss = 0.054822176694869995, batch-acc = 1.0
epoch = 194 step = 2730: train-loss = 0.08234139531850815, batch-acc = 1.0
epoch = 194, val-acc = 0.8454545454545455
Starting epoch 195 / 300
epoch = 195 step = 2740: train-loss = 0.042406871914863586, batch-acc = 1.0
epoch = 195, val-acc = 0.8181818181818182
Starting epoch 196 / 300
epoch = 196 step = 2750: train-loss = 0.09741330146789551, batch-acc = 1.0
epoch = 196, val-acc = 0.8454545454545455
Starting epoch 197 / 300
epoch = 197 step = 2760: train-loss = 0.044609468430280685, batch-acc 

KeyboardInterrupt: 

In [ ]:
    plt.plot(acc_list)
    plt.plot(acc_list, marker="*", linewidth=3, linestyle="--", color="orange")
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.title("model = {0}, base_lr = {1}, batch_size = {2}".format("senet_3", BASE_LR1, BATCH_SIZE))
    plt.show()

In [ ]:
# sha